In [ ]:
import requests
import pandas as pd
import time
from tabulate import tabulate
import numpy as np
import yfinance as yf

In [ ]:
def solicitar_tickers():
    print("Bienvenido al valuador por múltiplos comparables.")
    ticker_objetivo = input("Ingrese el ticker de la empresa que desea valuar: ").strip().upper()
    tickers_comparables = []
    print('\nIngrese los tickers de las empresas comparables uno por uno. Presione Enter sin escribir nada para terminar (mínimo 1).')
    while True:
        ticker = input(f"Ingrese ticker comparable #{len(tickers_comparables)+1}: ").strip()
        if ticker == "":
            if len(tickers_comparables) >= 1:
                break
            else:
                print("Debe ingresar al menos un ticker comparable.")
                continue
        tickers_comparables.append(ticker.upper())
    print(f"\nTicker a valuar: {ticker_objetivo}")
    print(f"Tickers comparables: {', '.join(tickers_comparables)}")
    return ticker_objetivo, tickers_comparables

ticker_objetivo, tickers_comparables = solicitar_tickers()

In [ ]:
def obtener_ratios_yahoo_finance(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        # Calcular Price / Free Cash Flow (P/FCF)
        market_cap = info.get("marketCap", None)
        free_cash_flow = info.get("freeCashflow", None)
        try:
            pfcf = market_cap / free_cash_flow if (market_cap is not None and free_cash_flow not in [None, 0]) else None
        except Exception:
            pfcf = None

        ratios = {
            "Ticker": ticker,
            "EV / EBITDA": info.get("enterpriseToEbitda", None),
            "Price / Earnings (P/E)": info.get("trailingPE", None),
            "EV / Revenue": info.get("enterpriseToRevenue", None),
            "Price / Free Cash Flow (P/FCF)": pfcf,
            "Price / Book Value": info.get("priceToBook", None),
            "Return on Assets (ROA)": info.get("returnOnAssets", None),
            "Return on Equity (ROE)": info.get("returnOnEquity", None),
            "Net Profit Margin": info.get("profitMargins", None),
            "Dividend Yield": info.get("dividendYield", None)
        }
        return ratios
    except Exception as e:
        print(f"Error obteniendo datos para {ticker}: {e}")
        return {
            "Ticker": ticker,
            "EV / EBITDA": None,
            "Price / Earnings (P/E)": None,
            "EV / Revenue": None,
            "Price / Free Cash Flow (P/FCF)": None,
            "Price / Book Value": None,
            "Return on Assets (ROA)": None,
            "Return on Equity (ROE)": None,
            "Net Profit Margin": None,
            "Dividend Yield": None
        }

# Uso (no necesitas renombrar después)
ratios_objetivo = obtener_ratios_yahoo_finance(ticker_objetivo)
df_objetivo = pd.DataFrame([ratios_objetivo])

comparables_data = []
for ticker in tickers_comparables:
    comparables_data.append(obtener_ratios_yahoo_finance(ticker))

df_comparables = pd.DataFrame(comparables_data)

In [ ]:
print("Empresa objetivo:")
print(tabulate(df_objetivo, headers='keys', tablefmt='fancy_grid', showindex=False))

print("\nEmpresas comparables:")
print(tabulate(df_comparables, headers='keys', tablefmt='fancy_grid', showindex=False))

In [ ]:
# Selecciona columnas numéricas (excluyendo 'Ticker')
ratios_cols = [col for col in df_comparables.columns if col != "Ticker"]

# Calcular promedio de cada ratio
promedio = df_comparables[ratios_cols].apply(pd.to_numeric, errors='coerce').mean(axis=0)
fila_promedio = {col: promedio[col] for col in ratios_cols}
fila_promedio["Ticker"] = "Promedio de ratios"

# Multiplicar los valores de la empresa objetivo por el promedio
empresa_vals = df_objetivo.iloc[0][ratios_cols].apply(pd.to_numeric, errors='coerce')
fila_valor_coef = {col: empresa_vals[col] * promedio[col] for col in ratios_cols}
fila_valor_coef["Ticker"] = "Valor por coeficiente"

# Crear nuevo DataFrame solo con esas dos filas
df_resumen = pd.DataFrame([fila_promedio, fila_valor_coef], columns=df_comparables.columns)

# Mostrar con tabulate
print(tabulate(df_resumen, headers="keys", tablefmt="fancy_grid", showindex=False))

In [ ]:
# Buscar la fila "Valor por coeficiente"
fila_valor_coef = df_resumen[df_resumen["Ticker"] == "Valor por coeficiente"]

# Tomar solo las columnas numéricas (ignorando 'Ticker')
ratios_cols = [col for col in df_resumen.columns if col != "Ticker"]
valores = fila_valor_coef[ratios_cols].iloc[0].apply(pd.to_numeric, errors='coerce')

# Ponderaciones (deben coincidir los nombres con ratios_cols)
ponderaciones = {
    "EV / EBITDA": 0.25,
    "Price / Earnings (P/E)": 0.15,
    "EV / Revenue": 0.10,
    "Price / Free Cash Flow (P/FCF)": 0.20,
    "Price / Book Value": 0.05,
    "Return on Assets (ROA)": 0.05,
    "Return on Equity (ROE)": 0.05,
    "Net Profit Margin": 0.05,
    "Dividend Yield": 0.10
}

# Filtrar ponderaciones solo para las columnas presentes y no NaN
ratios_validos = [col for col in ratios_cols if not pd.isna(valores[col])]
ponderaciones_validas = [ponderaciones[col] for col in ratios_validos]
valores_validos = [valores[col] for col in ratios_validos]

# Normalizar ponderaciones si falta algún ratio
ponderaciones_normalizadas = [p / sum(ponderaciones_validas) for p in ponderaciones_validas]

# Calcular el valor ponderado
valor_empresa = sum(v * p for v, p in zip(valores_validos, ponderaciones_normalizadas))

print(f"El valor estimado de la empresa (ponderado) es: {valor_empresa:.2f}")